In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from numpy import cos, sin, arcsin, sqrt
from math import radians

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Feature Crosses


In [ ]:
def haversine(pick_lon,pick_lat,drop_lon,drop_lat):
    lon1 = pick_lon
    lat1 = pick_lat
    lon2 =drop_lon
    lat2 = drop_lat

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    km = 6367 * c

    return km

def calculateDistance(df):
  distance_list = []
  pick_lon = df['pick_lon'].values
  pick_lat = df['pick_lat'].values
  drop_lon = df['drop_lon'].values
  drop_lat = df['drop_lat'].values

  for i in range(len(pick_lon)):  
    dist = haversine(pick_lon[i],pick_lat[i],drop_lon[i],drop_lat[i])
    distance_list.append(dist)
  df['distance'] = distance_list
  return df

def new_feature_cross(df):
  fare_to_duration = []
  distance_to_fare = []
  distance_to_duration = []
  waiting_fare_rate = []
  pickup_time_bin = []

  fare = df['fare'].values
  duration = df['duration'].values
  meter_waiting = df['meter_waiting'].values
  meter_waiting_fare = df['meter_waiting_fare'].values
  distance = df['distance'].values
  pickup_time = df['pickup_time'].values

  for i in range(len(fare)):
    if fare[i]!=0:
      fare_to_duration.append((duration[i]-meter_waiting[i])/fare[i])
    else:
      fare_to_duration.append((duration[i]-meter_waiting[i])/0.1)
    
    if distance[i]!=0:
      distance_to_fare.append(fare[i]/distance[i])
      distance_to_duration.append((duration[i]-meter_waiting[i])/distance[i])
    else:
      distance_to_fare.append(fare[i]/0.1)
      distance_to_duration.append((duration[i]-meter_waiting[i])/0.1)
  
    if meter_waiting_fare[i]!=0 :
      waiting_fare_rate.append(meter_waiting[i]/meter_waiting_fare[i])
    else:
      waiting_fare_rate.append(meter_waiting[i]/1)
    
    if int(pickup_time[i].split(" ")[0].split("/")[1]) in [1,10,12,16,17,18,19,21,22,26,27]:
      pickup_time_bin.append(1)
    else:
      pickup_time_bin.append(0)
    
  df['fare_to_duration'] = fare_to_duration
  df['distance_to_fare'] = distance_to_fare
  df['distance_to_duration'] = distance_to_duration
  df['waiting_fare_rate'] = waiting_fare_rate
  df['pickup_time'] = pickup_time_bin

  return df

# Bais Feature Preprocessing

Fill the missing figures of additional_fare column using median


In [ ]:
features_df = pd.read_csv('/content/drive/My Drive/MLproject/train.csv',index_col='tripid')
features_df['additional_fare'] = features_df['additional_fare'].fillna(features_df['additional_fare'].median())
features_df

Insert missing figures using mean values of respective correct and incorrect data

In [ ]:
correct = []
incorrect = []

features_df = pd.read_csv('/content/drive/My Drive/MLproject/train.csv',index_col='tripid')
features_df['additional_fare'] = features_df['additional_fare'].fillna(features_df['additional_fare'].median())

for i in features_df.values:
  if i[-1]=='correct':
    correct.append(i)
  else:
    incorrect.append(i)
  
correct = pd.DataFrame(correct,columns=features_df.columns)
incorrect = pd.DataFrame(incorrect,columns=features_df.columns)

correct = correct.fillna(correct.mean())
incorrect = incorrect.fillna(incorrect.mean())
features_df = pd.concat([correct,incorrect])

label_df = features_df['label']
#del features_df['label']
features_df = calculateDistance(features_df)
features_df = new_feature_cross(features_df)

In [ ]:
features_df = features_df[['additional_fare','duration','meter_waiting_fare',	'meter_waiting', 'fare','distance','meter_waiting_till_pickup','fare_to_duration','distance_to_fare','distance_to_duration','waiting_fare_rate','pickup_time']]
X_train, X_eval, y_train, y_eval = train_test_split(features_df,label_df,test_size=0.33,shuffle=True,stratify=label_df,random_state=6)

# Model creation and Evaluation

In [ ]:
# define model
model = XGBClassifier()              
model.fit(X_train,y_train)
prediction = model.predict(X_eval)

f1_score(y_eval, prediction, average='macro')

0.8579503150057274

In [ ]:
# define model
model = XGBClassifier(n_estimators=200,base_score=0.1)              
model.fit(X_train,y_train)
prediction = model.predict(X_eval)

f1_score(y_eval, prediction, average='macro')

0.8711826755869999

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_eval,prediction)

array([[5057,   57],
       [ 176,  379]])

# Predictions for Test data

In [ ]:
model.fit(features_df,label_df)

test_df = pd.read_csv('/content/drive/My Drive/MLproject/test.csv',index_col='tripid')

test_df = calculateDistance(test_df)
test_df = new_feature_cross(test_df)

test_df = test_df[['additional_fare','duration','meter_waiting_fare',	'meter_waiting', 'fare','distance','meter_waiting_till_pickup','fare_to_duration','distance_to_fare','distance_to_duration','waiting_fare_rate']]
prediction = model.predict(test_df)
prediction

array(['correct', 'correct', 'correct', ..., 'correct', 'correct',
       'correct'], dtype=object)

In [ ]:
result = []
for i in prediction:
  if i=='correct':
    result.append(1)
  else:
    result.append(0)

submission_df = pd.read_csv('/content/drive/My Drive/MLproject/sample_submission.csv')
submission_df['prediction'] = result
submission_df.to_csv('/content/drive/My Drive/MLproject/new_xgboost003.csv',index=False)